## Movielens

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime

pd.set_option('display.float_format', lambda x : '{:,.2f}'.format(x))
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
if not ('sc' in locals() or 'sc' in globals()):
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
    
    conf = SparkConf()
    conf.setMaster('spark://spark-master:7077')
    conf.set('spark.executor.memory', '512m')
    conf.set('spark.app.name', 'basics')


    sc = SparkContext.getOrCreate(SparkContext(conf=conf))
    
    spark = SparkSession \
        .builder \
        .getOrCreate()

21/11/16 10:05:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Context
### Exercise 1
__We want to know the 25 most rated movie titles from this data. How many times a movie has been rated?__

#### Users
Users: This file name is kept as “u.user”, The columns in this file are:

```python
['user_id', 'age', 'sex', 'occupation', 'zip_code']
```

#### Ratings
Ratings: This file name is kept as “u.data”, The columns in this file are:

```python
['user_id', 'movie_id', 'rating', 'unix_timestamp']
```

#### Movies
Movies: This file name is kept as “u.item”, The columns in this file are:

```python
['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url', and 18 more columns.....]
```

#### Load data
The firs step is to see how the data looks like

More info: https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#dataframe-faqs

In [3]:
userRDD = sc.textFile("../data/u.user") 
userRDD.take(1)

['1|24|M|technician|85711']

In [6]:
df_user = spark.read.format("csv").options(header='false', delimiter = '|').load("../data/u.user")
df_user = df_user.toDF('user_id', 'age', 'sex', 'occupation', 'zip_code')

df_user.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [17]:
df_user.show(5)

+-------+---+---+----------+--------+
|user_id|age|sex|occupation|zip_code|
+-------+---+---+----------+--------+
|      1| 24|  M|technician|   85711|
|      2| 53|  F|     other|   94043|
|      3| 23|  M|    writer|   32067|
|      4| 24|  M|technician|   43537|
|      5| 33|  F|     other|   15213|
+-------+---+---+----------+--------+
only showing top 5 rows



In [4]:
ratingRDD = sc.textFile("../data/u.data") 
ratingRDD.take(1)

['196\t242\t3\t881250949']

In [11]:
df_rating = spark.read.format("csv").options(header='false', delimiter = '\t').load("../data/u.data")
df_rating = df_rating.toDF('user_id', 'movie_id', 'rating', 'unix_timestamp')

df_rating.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- movie_id: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- unix_timestamp: string (nullable = true)



In [18]:
df_rating.show(5)

+-------+--------+------+--------------+
|user_id|movie_id|rating|unix_timestamp|
+-------+--------+------+--------------+
|    196|     242|     3|     881250949|
|    186|     302|     3|     891717742|
|     22|     377|     1|     878887116|
|    244|      51|     2|     880606923|
|    166|     346|     1|     886397596|
+-------+--------+------+--------------+
only showing top 5 rows



In [27]:
movieRDD = sc.textFile("../data/u.item") 
movieRDD.take(1)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0']

In [28]:
df_movie = spark.read.format("csv").options(header='false', delimiter = '|').load("../data/u.item")
df_movie = df_movie.toDF('movie_id', 'movie_title' ,'release date','video_release date', 'IMDb_URL', 'unknown', 'Action', 
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western')

df_movie.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- release date: string (nullable = true)
 |-- video_release date: string (nullable = true)
 |-- IMDb_URL: string (nullable = true)
 |-- unknown: string (nullable = true)
 |-- Action: string (nullable = true)
 |-- Adventure: string (nullable = true)
 |-- Animation: string (nullable = true)
 |-- Children's: string (nullable = true)
 |-- Comedy: string (nullable = true)
 |-- Crime: string (nullable = true)
 |-- Documentary: string (nullable = true)
 |-- Drama: string (nullable = true)
 |-- Fantasy: string (nullable = true)
 |-- Film-Noir: string (nullable = true)
 |-- Horror: string (nullable = true)
 |-- Musical: string (nullable = true)
 |-- Mystery: string (nullable = true)
 |-- Romance: string (nullable = true)
 |-- Sci-Fi: string (nullable = true)
 |-- Thriller: string (nullable = true)
 |-- War: string (nullable = true)
 |-- Western: string (nullable = true)



In [29]:
df_movie.limit(5).toPandas()

,movie_id,movie_title,release date,video_release date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


#### Merge


In [30]:
rating_movie = df_rating.join(df_movie.select(['movie_id', 'movie_title']), 
                              df_movie.movie_id == df_rating.movie_id, 
                              how='left')

In [34]:
rating_movie.show(5, truncate=True)

+-------+--------+------+--------------+--------+--------------------+
|user_id|movie_id|rating|unix_timestamp|movie_id|         movie_title|
+-------+--------+------+--------------+--------+--------------------+
|    196|     242|     3|     881250949|     242|        Kolya (1996)|
|    186|     302|     3|     891717742|     302|L.A. Confidential...|
|     22|     377|     1|     878887116|     377| Heavyweights (1994)|
|    244|      51|     2|     880606923|      51|Legends of the Fa...|
|    166|     346|     1|     886397596|     346| Jackie Brown (1997)|
+-------+--------+------+--------------+--------+--------------------+
only showing top 5 rows



Use the RDD in previous step to create (movie, 1) tuple pair RDD

In [41]:
from pyspark.sql.functions import count, col

(rating_movie
 .groupby(['movie_title'])
 .agg(count(col('rating')).alias('counts'))
 .sort(col('counts').desc())
 .show(5, truncate=True))

+--------------------+------+
|         movie_title|counts|
+--------------------+------+
|    Star Wars (1977)|   583|
|      Contact (1997)|   509|
|        Fargo (1996)|   508|
|Return of the Jed...|   507|
|    Liar Liar (1997)|   485|
+--------------------+------+
only showing top 5 rows



### Exercise 2

__Now we want to find the most highly rated 25 movies using the same dataset. We actually want only those movies which have been rated at least 100 times.__

In [45]:
from pyspark.sql.functions import sum

(rating_movie
 .groupby(['movie_title'])
 .agg(count(col('rating')).alias('counts'),
      sum(col('rating')).alias('sum_rate'))
 .filter(col('counts') >= 100)
 .withColumn('mean_rate', col('sum_rate') / col('counts'))
 .sort(col('mean_rate').desc())
 .show(5))

+--------------------+------+--------+-----------------+
|         movie_title|counts|sum_rate|        mean_rate|
+--------------------+------+--------+-----------------+
|Close Shave, A (1...|   112|   503.0|4.491071428571429|
|Schindler's List ...|   298|  1331.0|4.466442953020135|
|Wrong Trousers, T...|   118|   527.0|4.466101694915254|
|   Casablanca (1942)|   243|  1083.0| 4.45679012345679|
|Shawshank Redempt...|   283|  1258.0|4.445229681978798|
+--------------------+------+--------+-----------------+
only showing top 5 rows

